
Coinbase Pro API Test
The sandbox version of the coinbase Pro is at:

https://public.sandbox.pro.coinbase.com

Cbpro Python library repository:

https://github.com/danpaquin/coinbasepro-python

API url (sandbox):

https://api-public.sandbox.pro.coinbase.com

Websocket url (sandbox):

wss://ws-feed-public.sandbox.pro.coinbase.com

In [ ]:
import sys
sys.version


In [1]:
%config Completer.use_jedi = False       #without this line, autocomplete seemed to not be working, 
#so I am putting it here. source: https://stackoverflow.com/questions/40536560/ipython-and-jupyter-autocomplete-not-working

In [2]:
import pandas as pd
import cbpro
from configs import (api_secret,
                              api_key, 
                              api_pass)
print(api_key, api_secret, api_pass)

cd66ce5a639f411d9052866646919394 x+zajwcASNpyIZUAPYUbDCzvyxXgz7xf5kRxseFLwdRhO1f4NXshoddKheMRSomH7W0dJUjGlbtt5pxAORYgRg== notSandbox


# WebsocketClient Class
Here we inherit from the WebsocketClient class and overload methods on_open(), on_message(), and on_close()

In [ ]:
class TextWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url           = 'wss://ws-feed-public.sandbox.pro.coinbase.com'
        self.message_count = 0
    
    def on_message(self,msg):
        self.message_count += 1
        msg_type = msg.get('type',None)
        if msg_type == 'ticker':
            time_val   = msg.get('time',('-'*27))
            price_val  = msg.get('price',None)
            if price_val is not None:
                price_val = float(price_val)
            product_id = msg.get('product_id',None)
            
            print(f"{time_val:30} \
                {price_val:.3f} \
                {product_id}\tchannel type:{msg_type}")
    
    def on_close(self):
        print(f"<---Websocket connection closed--->\n\tTotal messages: {self.message_count}")
        

In [ ]:
stream = TextWebsocketClient(products=['BTC-USD'],channels=['ticker'])
stream.start()

In [ ]:
stream.close()

In [30]:
#url = 'https://api.pro.coinbase.com'

url = 'https://api-public.sandbox.pro.coinbase.com'

client = cbpro.AuthenticatedClient(
    api_key,
    api_secret,
    api_pass, 
    api_url=url)

In [31]:
accounts = client.get_accounts()


In [32]:
print(accounts)

{'message': 'Invalid API Key'}


In [22]:
for acc in accounts:
    currency = acc.get('currency')
    if currency=='BTC':
        acc_id = acc.get('id')

print(acc_id)


047b04e7-6839-43e7-a1c0-54ae5bc6a477


In [25]:
acc_history = client.get_account_history(acc_id)

print(acc_history)

<generator object PublicClient._send_paginated_message at 0x0000019E62D3B740>


In [26]:
import json

for hist in acc_history:
    print(json.dumps(hist,indent=1))

In [27]:
client.place_market_order(product_id='ADA-USD',side='buy',size=1)

{'message': 'Insufficient funds'}

In [28]:
payment_methods = client.get_payment_methods()
print(payment_methods)

{'message': 'Forbidden'}


In [ ]:
for method in payment_methods:
    currency  = method.get('currency',None)
    if currency.upper() == 'USD':
        method_id = method.get('id',None) 
    elif currency is None:
        continue

print(f"Currency is `{currency}`\n")

In [ ]:
client.deposit(
    amount=30,
    currency=currency,
    payment_method_id=method_id
)